<a href="https://colab.research.google.com/github/Cody9494/LEGALSKEPSIS-DATA/blob/main/LAWSKPEPSIS_STEP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install pandas numpy tqdm faiss-cpu sentence-transformers rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

import pandas as pd

path = "/content/drive/MyDrive/LAWSKEPSIS/df_master_clean.parquet"

df_master = pd.read_parquet(path)

print(df_master.shape)
df_master


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(27264, 9)


,act_id,act_type,reg_number,title,date_signed,legal_basis,recitals,articles,signatory
0,2023/207,Commission Delegated Regulation,2023/207,(EU) 2023/207 Commission Delegated Regulation ...,20221124,[Having regard to the Treaty on the Functionin...,[Annex VI to Regulation (EU) 2018/848 sets out...,[{'article_body': 'Annex VI to Regulation (EU)...,"[For the Commission ITALIC, The President ITAL..."
1,2025/1003,Commission Delegated Regulation,2025/1003,(EU) 2025/1003 Commission Delegated Regulation...,20250124,[Having regard to the Treaty on the Functionin...,[Identifying reference data should enable mark...,[{'article_body': '1. From 1 September 2026 20...,"[For the Commission ITALIC, The President ITAL..."
2,2016/182,Commission Implementing Regulation,2016/182,(EU) 2016/182 Commission Implementing Regulati...,20160211,[Having regard to the Treaty on the Functionin...,[The approval of the active substance pyrafluf...,[{'article_body': 'The approval of the active ...,"[For the Commission ITALIC, The President ITAL..."
3,2010/222,Commission Decision,2010/222,Commission Decision of 19 April 2010 20100419 ...,20100419,[Having regard to the Treaty on the Functionin...,[The European Union has stated goals in the en...,[{'article_body': 'The European Commission (he...,"[For the Commission ITALIC, Günther Oettinger ..."
4,2021/1743,Council Decision,2021/1743,(EU) 2021/1743 Council Decision of 28 Septembe...,20210928,[Having regard to the Treaty on the Functionin...,[The Treaty establishing the Transport Communi...,[{'article_body': 'The position to be taken on...,"[For the Council ITALIC, The President ITALIC,..."
...,...,...,...,...,...,...,...,...,...
27259,2008/170,Commission Decision,2008/170,Commission Decision of 27 February 2008 200802...,20080227,[Having regard to the Treaty establishing the ...,"[Under Article 75 of Regulation (EC, Euratom) ...",[{'article_body': '1. The work plan for 2008 s...,"[For the Commission ITALIC, Markos Kyprianou U..."
27260,2014/142,Council Implementing Decision,2014/142,2014/142/CFSP Council Implementing Decision of...,20140314,[Having regard to the Treaty on European Union...,"[On, the Council adopted Decision 2011/486/CFS...",[{'article_body': 'The Annex to Decision 2011/...,"[For the Council ITALIC, The President ITALIC,..."
27261,2023/2821,,,DRAFT DECISION OF THE EEA JOINT COMMITTEE No …...,20231204,[Having regard to the Agreement on the Europea...,[Regulation (EU) 2019/1149 of the European Par...,[{'article_body': 'Annex V to the EEA Agreemen...,"[For the EEA Joint Committee ITALIC, The Presi..."
27262,2023/2821,Council Decision,2023/2821,(EU) 2023/2821 Council Decision of 4 December ...,20231204,[Having regard to the Treaty on the Functionin...,[The Agreement on the European Economic Area (...,"[{'article_body': 'The position to be adopted,...","[For the Council ITALIC, The President ITALIC,..."


In [ ]:
KEEP_COLS = [
    "act_id",
    "act_type",
    "reg_number",
    "title",
    "date_signed",
    "legal_basis",
    "recitals",
    "articles",
    "signatory",
]


In [ ]:
for c in KEEP_COLS:
    if c not in df_master.columns:
        df_master[c] = pd.NA

df_master = df_master[KEEP_COLS].copy()
print("After column selection:", df_master.shape)


After column selection: (27264, 9)


In [ ]:
import re
import pandas as pd

def _norm_space(s: str) -> str:
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    s = re.sub(r"[ \t]+", " ", s)
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s.strip()

def _ensure_sentence_end(s: str) -> str:
    s = s.strip()
    if not s:
        return s
    # Αν τελειώνει ήδη σωστά, άστο.
    if re.search(r"[.!?…]$", s):
        return s
    # Αν τελειώνει με κόμμα/άνω τελεία/άνω-κάτω, κάνε το τελεία.
    if re.search(r"[,;:]\s*$", s):
        s = re.sub(r"[,;:]\s*$", ".", s)
        return s
    # Default: βάλε τελεία
    return s + "."

def _clean_punct(s: str) -> str:
    # Αφαιρεί διπλά σημεία στίξης που εμφανίζονται από joins
    s = re.sub(r"\s+\.", ".", s)
    s = re.sub(r"\.\.+", ".", s)
    s = re.sub(r",,+", ",", s)
    s = re.sub(r"\s+,", ",", s)
    s = re.sub(r"\s+\)", ")", s)
    s = re.sub(r"\(\s+", "(", s)
    return s

def format_block(label: str, value) -> str:
    """
    value μπορεί να είναι:
    - string
    - list[str]
    - list[dict] (π.χ. articles)
    - dict
    - list of mixed
    """
    if value is None or (isinstance(value, float) and pd.isna(value)):
        return ""

    # STRING
    if isinstance(value, str):
        v = _norm_space(value)
        if not v:
            return ""
        return f"{label}:\n{v}\n"

    # LIST
    if isinstance(value, list):
        items = []
        for i, it in enumerate(value, start=1):
            if it is None or (isinstance(it, float) and pd.isna(it)):
                continue

            # item is dict (π.χ. article entries)
            if isinstance(it, dict):
                # Προσπάθησε να βγάλεις “χρήσιμα” κλειδιά με σταθερή σειρά
                # (προσαρμόζεται εύκολα αν θες άλλα keys)
                preferred_keys = ["article_number", "article_title", "article_body", "title", "body", "text"]
                lines = []
                for k in preferred_keys:
                    if k in it and it[k]:
                        lines.append(f"{k}: {_norm_space(str(it[k]))}")
                if not lines:
                    # fallback: όλο το dict
                    lines.append(_norm_space(str(it)))
                item_txt = " | ".join(lines)
                item_txt = _ensure_sentence_end(item_txt)
                items.append(f"{i}. {item_txt}")
                continue

            # item is string / other
            item_txt = _norm_space(str(it))
            item_txt = _ensure_sentence_end(item_txt)
            items.append(f"{i}. {item_txt}")

        if not items:
            return ""
        return f"{label}:\n" + "\n".join(items) + "\n"

    # DICT (όχι list)
    if isinstance(value, dict):
        # Σταθερό formatting key: value
        lines = []
        for k, v in value.items():
            if v is None or (isinstance(v, float) and pd.isna(v)):
                continue
            v_txt = _norm_space(str(v))
            if v_txt:
                lines.append(f"- {k}: {v_txt}")
        if not lines:
            return ""
        return f"{label}:\n" + "\n".join(lines) + "\n"

    # fallback: scalar
    v = _norm_space(str(value))
    if not v:
        return ""
    return f"{label}: {v}\n"


def build_embedding_text_row(row: pd.Series, columns_order: list[str]) -> str:
    """
    Χτίζει embedding_text από συγκεκριμένη σειρά columns.
    """
    parts = []
    for col in columns_order:
        if col not in row.index:
            continue
        val = row[col]
        block = format_block(col, val)
        if block:
            parts.append(block)

    text = "\n".join(parts)
    text = _clean_punct(_norm_space(text))
    return text


def apply_embedding_text(df: pd.DataFrame, columns_order: list[str],
                         out_col: str = "embedding_text_v2",
                         len_col: str = "embedding_text_v2_len") -> pd.DataFrame:
    df[out_col] = df.apply(lambda r: build_embedding_text_row(r, columns_order), axis=1)
    df[len_col] = df[out_col].str.len()
    return df


In [ ]:
columns_order = [
    "act_id",
    "act_type",
    "reg_number",
    "title",
    "date_signed",
    "legal_basis",
    "recitals",
    "articles",
    "signatory",
]
df_master = apply_embedding_text(df_master, columns_order, out_col="embedding_text_v2")


In [ ]:
# df_master = apply_embedding_text(df_master, KEEP_COLS)


In [ ]:
pd.set_option("display.max_colwidth", None)

sample_idx = df_master.sample(3, random_state=42).index.tolist()
for idx in sample_idx:
    print("="*120)
    print("ROW INDEX:", idx)
    print("="*120)
    print(df_master.loc[idx, "embedding_text_v2"])
    print("\nLEN:", df_master.loc[idx, "embedding_text_v2_len"])


ROW INDEX: 15245
act_id:
2013/473

act_type:
Regulation of the European Parliament and of the Council

title:
(EU) No 473/2013 Regulation of the European Parliament and of the Council of 21 May 2013 20130521 on common provisions for monitoring and assessing draft budgetary plans and ensuring the correction of excessive deficit of the Member States in the euro area

date_signed:
20130521

legal_basis: ['Having regard to the Treaty on the Functioning of the European Union, and in particular Article 136 in combination with Article 121(6) thereof,'
 'Having regard to the proposal from the European Commission,'
 'After transmission of the draft legislative act to the national parliaments,'
 'Having regard to the opinion of the European Central Bank,. OJ C 141, 17.5.2012, p. 7 C 20120517 141 7'
 'Acting in accordance with the ordinary legislative procedure, Position of the European Parliament of (not yet published in the Official Journal) and decision of the Council of. 12 March 2013 2013031

In [ ]:
df_master

,act_id,act_type,reg_number,title,date_signed,legal_basis,recitals,articles,signatory
0,2023/207,Commission Delegated Regulation,2023/207,(EU) 2023/207 Commission Delegated Regulation ...,20221124,[Having regard to the Treaty on the Functionin...,[Annex VI to Regulation (EU) 2018/848 sets out...,[{'article_body': 'Annex VI to Regulation (EU)...,"[For the Commission ITALIC, The President ITAL..."
1,2025/1003,Commission Delegated Regulation,2025/1003,(EU) 2025/1003 Commission Delegated Regulation...,20250124,[Having regard to the Treaty on the Functionin...,[Identifying reference data should enable mark...,[{'article_body': '1. From 1 September 2026 20...,"[For the Commission ITALIC, The President ITAL..."
2,2016/182,Commission Implementing Regulation,2016/182,(EU) 2016/182 Commission Implementing Regulati...,20160211,[Having regard to the Treaty on the Functionin...,[The approval of the active substance pyrafluf...,[{'article_body': 'The approval of the active ...,"[For the Commission ITALIC, The President ITAL..."
3,2010/222,Commission Decision,2010/222,Commission Decision of 19 April 2010 20100419 ...,20100419,[Having regard to the Treaty on the Functionin...,[The European Union has stated goals in the en...,[{'article_body': 'The European Commission (he...,"[For the Commission ITALIC, Günther Oettinger ..."
4,2021/1743,Council Decision,2021/1743,(EU) 2021/1743 Council Decision of 28 Septembe...,20210928,[Having regard to the Treaty on the Functionin...,[The Treaty establishing the Transport Communi...,[{'article_body': 'The position to be taken on...,"[For the Council ITALIC, The President ITALIC,..."
...,...,...,...,...,...,...,...,...,...
27259,2008/170,Commission Decision,2008/170,Commission Decision of 27 February 2008 200802...,20080227,[Having regard to the Treaty establishing the ...,"[Under Article 75 of Regulation (EC, Euratom) ...",[{'article_body': '1. The work plan for 2008 s...,"[For the Commission ITALIC, Markos Kyprianou U..."
27260,2014/142,Council Implementing Decision,2014/142,2014/142/CFSP Council Implementing Decision of...,20140314,[Having regard to the Treaty on European Union...,"[On, the Council adopted Decision 2011/486/CFS...",[{'article_body': 'The Annex to Decision 2011/...,"[For the Council ITALIC, The President ITALIC,..."
27261,2023/2821,,,DRAFT DECISION OF THE EEA JOINT COMMITTEE No …...,20231204,[Having regard to the Agreement on the Europea...,[Regulation (EU) 2019/1149 of the European Par...,[{'article_body': 'Annex V to the EEA Agreemen...,"[For the EEA Joint Committee ITALIC, The Presi..."
27262,2023/2821,Council Decision,2023/2821,(EU) 2023/2821 Council Decision of 4 December ...,20231204,[Having regard to the Treaty on the Functionin...,[The Agreement on the European Economic Area (...,"[{'article_body': 'The position to be adopted,...","[For the Council ITALIC, The President ITALIC,..."


In [ ]:
df_master["act_id"] = df_master["act_id"].astype("string")

missing_act_id_mask = df_master["act_id"].isna() | (df_master["act_id"].str.strip() == "")
print("Rows with missing act_id:", int(missing_act_id_mask.sum()))

df_master = df_master[~missing_act_id_mask].copy()
print("After dropping missing act_id:", df_master.shape)


Rows with missing act_id: 0
After dropping missing act_id: (27264, 9)


In [ ]:
def ensure_list(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return []
    if isinstance(x, list):
        return x
    # αν είναι string που μοιάζει με list, ΔΕΝ το eval-άρουμε για ασφάλεια.
    # το κρατάμε ως single-item list
    return [str(x)]

def ensure_articles(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return []
    if isinstance(x, list):
        # αναμένουμε list of dicts
        out = []
        for item in x:
            if isinstance(item, dict):
                out.append(item)
            else:
                out.append({"article_number": "", "article_title": "", "article_body": str(item)})
        return out
    # fallback
    return [{"article_number": "", "article_title": "", "article_body": str(x)}]

for col in ["legal_basis", "recitals", "signatory"]:
    df_master[col] = df_master[col].apply(ensure_list)

df_master["articles"] = df_master["articles"].apply(ensure_articles)

# βασικά strings
for col in ["act_type", "reg_number", "title", "date_signed"]:
    df_master[col] = df_master[col].astype("string").fillna("")


In [ ]:
import re
_ws_re = re.compile(r"\s+")

def clean_ws(s: str) -> str:
    s = "" if s is None else str(s)
    s = s.replace("\u00a0", " ")          # nbsp
    s = _ws_re.sub(" ", s).strip()
    return s

def format_section_list(header: str, items: list) -> str:
    if not items:
        return ""
    lines = [header]
    for i, it in enumerate(items, start=1):
        t = clean_ws(it)
        if t:
            lines.append(f"{i}. {t}")
    return "\n".join(lines)

def format_articles(articles: list) -> str:
    if not articles:
        return ""
    blocks = ["ARTICLES:"]
    for a in articles:
        num = clean_ws(a.get("article_number", ""))
        ttl = clean_ws(a.get("article_title", ""))
        body = clean_ws(a.get("article_body", ""))
        # Μηδενική απώλεια: δεν κόβουμε body
        head = f"Article {num}".strip() if num else "Article"
        if ttl:
            head = f"{head} — {ttl}"
        blocks.append(head)
        if body:
            blocks.append(body)
        blocks.append("")  # κενή γραμμή ανά άρθρο
    return "\n".join(blocks).strip()

def build_embedding_text(row) -> str:
    act_type = clean_ws(row["act_type"])
    title = clean_ws(row["title"])
    reg_number = clean_ws(row["reg_number"])
    act_id = clean_ws(row["act_id"])
    date_signed = clean_ws(row["date_signed"])

    # Sections
    legal_basis_txt = format_section_list("LEGAL BASIS:", row["legal_basis"])
    recitals_txt = format_section_list("RECITALS:", row["recitals"])
    signatory_txt = format_section_list("SIGNATORY:", row["signatory"])
    articles_txt = format_articles(row["articles"])

    header_lines = []
    header_lines.append(f"ACT_ID: {act_id}" if act_id else "ACT_ID:")
    if act_type: header_lines.append(f"ACT_TYPE: {act_type}")
    if date_signed: header_lines.append(f"DATE_SIGNED: {date_signed}")
    if reg_number: header_lines.append(f"REG_NUMBER: {reg_number}")
    if title: header_lines.append(f"TITLE: {title}")

    parts = [
        "\n".join(header_lines).strip(),
        legal_basis_txt,
        recitals_txt,
        articles_txt,
        signatory_txt,
    ]
    # κρατάμε μόνο non-empty parts, με καθαρές αποστάσεις
    text = "\n\n".join([p for p in parts if p and p.strip()])

    return text

df_master["embedding_text"] = df_master.apply(build_embedding_text, axis=1)


In [ ]:
def ends_with_bad_connector(s: str) -> bool:
    s = clean_ws(s).lower()
    if not s:
        return True
    bad_endings = (" and", " or", " of", " with", " per", " to", " in", " on", " for", " by", " from")
    if s.endswith(","):
        return True
    return any(s.endswith(be) for be in bad_endings)

df_master["schema_ok"] = df_master["act_id"].str.strip().ne("")
df_master["embedding_text_len"] = df_master["embedding_text"].str.len().astype("int64")

df_master["flag_text_empty"] = df_master["embedding_text_len"].eq(0)
df_master["flag_bad_ending"] = df_master["embedding_text"].apply(ends_with_bad_connector)

print(df_master[["schema_ok","flag_text_empty","flag_bad_ending"]].value_counts().head(10))
print("Rows flagged (bad ending):", int(df_master["flag_bad_ending"].sum()))


schema_ok  flag_text_empty  flag_bad_ending
True       False            False              27264
Name: count, dtype: int64
Rows flagged (bad ending): 0


In [ ]:
pd.set_option("display.max_colwidth", None)

sample_idx = df_master.sample(3, random_state=42).index.tolist()
for idx in sample_idx:
    print("="*120)
    print("ROW INDEX:", idx)
    print("="*120)
    print(df_master.loc[idx, "embedding_text"])
    print("\nLEN:", df_master.loc[idx, "embedding_text_len"])


ROW INDEX: 15245
ACT_ID: 2013/473
ACT_TYPE: Regulation of the European Parliament and of the Council
DATE_SIGNED: 20130521
TITLE: (EU) No 473/2013 Regulation of the European Parliament and of the Council of 21 May 2013 20130521 on common provisions for monitoring and assessing draft budgetary plans and ensuring the correction of excessive deficit of the Member States in the euro area

LEGAL BASIS:
1. ['Having regard to the Treaty on the Functioning of the European Union, and in particular Article 136 in combination with Article 121(6) thereof,' 'Having regard to the proposal from the European Commission,' 'After transmission of the draft legislative act to the national parliaments,' 'Having regard to the opinion of the European Central Bank,. OJ C 141, 17.5.2012, p. 7 C 20120517 141 7' 'Acting in accordance with the ordinary legislative procedure, Position of the European Parliament of (not yet published in the Official Journal) and decision of the Council of. 12 March 2013 20130312 13

In [ ]:
import re
import ast

def looks_like_py_list(s: str) -> bool:
    if not isinstance(s, str):
        return False
    t = s.strip()
    return t.startswith("[") and t.endswith("]") and ("'" in t or '"' in t)

def flatten_py_list_text(s: str, sep="\n- ") -> str:
    """
    Converts a python-list-as-text into clean text bullets.
    If parse fails, falls back to conservative regex cleanup.
    """
    if not isinstance(s, str):
        return s
    t = s.strip()

    if looks_like_py_list(t):
        try:
            obj = ast.literal_eval(t)
            if isinstance(obj, list):
                items = []
                for x in obj:
                    if x is None:
                        continue
                    items.append(str(x).strip())
                if not items:
                    return ""
                return "- " + sep.join(items).lstrip("- ").strip()
        except Exception:
            pass

    # fallback: remove only outer quotes around apparent items: '...'
    # safe: does NOT touch apostrophes inside words
    return re.sub(r"(^|[\s\[\(])'([^']+)'(?=[\s\],\)]|$)", r"\1\2", s)

def clean_embedding_text_quotes(text: str) -> str:
    if not isinstance(text, str):
        return text

    x = text

    # flatten any inline py-list blocks after "1." patterns: "1. ['...','...']"
    # we replace only the bracketed part
    def repl(m):
        prefix = m.group(1)
        block = m.group(2)
        return prefix + flatten_py_list_text(block)

    x = re.sub(r"(\n\d+\.\s*)(\[[\s\S]*?\])", repl, x)

    # also catch cases like "Article\n[{'...'} {'...'}]" if needed later (different structure)

    return x


In [ ]:
# 1) Δημιουργεί νέα καθαρισμένη έκδοση (προτείνεται για safety)
df_master["embedding_text_clean"] = df_master["embedding_text"].apply(clean_embedding_text_quotes)


In [ ]:
sample_idx = df_master.sample(3, random_state=42).index.tolist()
for idx in sample_idx:
    print("="*120)
    print("ROW INDEX:", idx)
    print("="*120)
    print("BEFORE:\n", df_master.loc[idx, "embedding_text"][:2000])
    print("\nAFTER:\n", df_master.loc[idx, "embedding_text_clean"][:2000])


ROW INDEX: 15245
BEFORE:
 ACT_ID: 2013/473
ACT_TYPE: Regulation of the European Parliament and of the Council
DATE_SIGNED: 20130521
TITLE: (EU) No 473/2013 Regulation of the European Parliament and of the Council of 21 May 2013 20130521 on common provisions for monitoring and assessing draft budgetary plans and ensuring the correction of excessive deficit of the Member States in the euro area

LEGAL BASIS:
1. ['Having regard to the Treaty on the Functioning of the European Union, and in particular Article 136 in combination with Article 121(6) thereof,' 'Having regard to the proposal from the European Commission,' 'After transmission of the draft legislative act to the national parliaments,' 'Having regard to the opinion of the European Central Bank,. OJ C 141, 17.5.2012, p. 7 C 20120517 141 7' 'Acting in accordance with the ordinary legislative procedure, Position of the European Parliament of (not yet published in the Official Journal) and decision of the Council of. 12 March 2013 20

In [ ]:
df

,internalId,name,description,legislationIdentifier,legislationDate,legislationType,legislationPassedBy,legalBasis_text,signatory_text,articles_raw,normalized_text
0,2023/207,(EU) 2023/207 of amending Regulation (EU) 2018...,Annex VI to Regulation (EU) 2018/848 sets out ...,2023/207,2022-11-24,Commission Delegated Regulation,European Parliament and the Council of the Eur...,Having regard to the Treaty on the Functioning...,Ursula,"[{'article_number': '1', 'article_title': '', ...",(eu) 2023/207 of amending regulation (eu) 2018...
1,2025/1003,(EU) 2025/1003 of supplementing Regulation (EU...,Identifying reference data should enable marke...,2025/1003,2025-01-24,Commission Delegated Regulation,European Parliament and the Council of the Eur...,Having regard to the Treaty on the Functioning...,Ursula,"[{'article_number': '1', 'article_title': 'Ide...",(eu) 2025/1003 of supplementing regulation (eu...
2,2016/182,(EU) 2016/182 of renewing the approval of the ...,The approval of the active substance pyraflufe...,2016/182,2016-02-11,Commission Implementing Regulation,European Parliament and the Council of the Eur...,Having regard to the Treaty on the Functioning...,Jean-Claude,"[{'article_number': '1', 'article_title': 'Ren...",(eu) 2016/182 of renewing the approval of the ...
3,2010/222,of on the European Commission becoming a partn...,The European Union has stated goals in the ene...,2010/222,2010-04-19,Commission Decision,European Commission,Having regard to the Treaty on the Functioning...,Günther,"[{'article_number': '1', 'article_title': '', ...",of on the european commission becoming a partn...
4,2021/1743,(EU) 2021/1743 of on the position to be taken ...,The Treaty establishing the Transport Communit...,2021/1743,2021-09-28,Council Decision,Council of the European Union,Having regard to the Treaty on the Functioning...,S.,"[{'article_number': '1', 'article_title': '', ...",(eu) 2021/1743 of on the position to be taken ...
...,...,...,...,...,...,...,...,...,...,...,...
27295,2008/170,of on the adoption of the work plan for 2008 f...,"Under Article 75 of Regulation (EC, Euratom) N...",2008/170,2008-02-27,Commission Decision,European Commission,Having regard to the Treaty establishing the E...,Markos,"[{'article_number': '1', 'article_title': '', ...",of on the adoption of the work plan for 2008 f...
27296,2014/142,2014/142/CFSP of implementing Decision 2011/48...,"On , the Council adopted Decision 2011/486/CFS...",2014/142,2014-03-14,Council Implementing Decision,European Union,"Having regard to the Treaty on European Union,...",M.,"[{'article_number': '1', 'article_title': '', ...",2014/142/cfsp of implementing decision 2011/48...
27297,NaN,DRAFT DECISION OF THE EEA JOINT COMMITTEE No …...,Regulation (EU) 2019/1149 of the European Parl...,NaN,2023-12-04,NaN,European Union,Having regard to the Agreement on the European...,NaN,"[{'article_number': '1', 'article_title': '', ...",draft decision of the eea joint committee no …...
27298,2023/2821,(EU) 2023/2821 of on the position to be adopte...,The Agreement on the European Economic Area (t...,2023/2821,2023-12-04,Council Decision,Council of the European Union,Having regard to the Treaty on the Functioning...,F.,"[{'article_number': '1', 'article_title': '', ...",(eu) 2023/2821 of on the position to be adopte...


In [ ]:
df.dtypes

,0
internalId,object
name,object
description,object
legislationIdentifier,object
legislationDate,object
legislationType,object
legislationPassedBy,object
legalBasis_text,object
signatory_text,object
articles_raw,object


In [ ]:
import pandas as pd
import numpy as np

required_cols = [
    "internalId","name","description","legislationIdentifier","legislationDate",
    "legislationType","legislationPassedBy","legalBasis_text","signatory_text",
    "articles_raw","normalized_text"
]

missing = [c for c in required_cols if c not in df.columns]
print("Missing required cols:", missing)

# Basic null/empty checks
def empty_ratio(s: pd.Series) -> float:
    return (s.isna() | (s.astype(str).str.strip() == "")).mean()

for c in ["name","description","legalBasis_text","articles_raw","normalized_text"]:
    print(c, "empty_ratio =", round(empty_ratio(df[c]), 4))

# Length diagnostics (characters)
for c in ["description","legalBasis_text","normalized_text"]:
    lens = df[c].astype(str).str.len()
    print(f"\n{c} length stats (chars):")
    print(lens.describe(percentiles=[.5,.75,.9,.95,.99]).round(0))

# Duplicates by legislationIdentifier (συχνό σε consolidated/amending variants)
dup_count = df.duplicated(subset=["legislationIdentifier"]).sum()
print("\nDuplicated legislationIdentifier rows:", dup_count)

# Quick check: how many have huge text (likely annex-heavy)
norm_len = df["normalized_text"].astype(str).str.len()
print("\nNormalized_text > 50k chars:", (norm_len > 50000).sum())
print("Normalized_text > 200k chars:", (norm_len > 200000).sum())


Missing required cols: []
name empty_ratio = 0.0014
description empty_ratio = 0.0917
legalBasis_text empty_ratio = 0.0014
articles_raw empty_ratio = 0.0
normalized_text empty_ratio = 0.0003

description length stats (chars):
count    27300.0
mean       817.0
std        324.0
min          3.0
50%       1003.0
75%       1003.0
90%       1003.0
95%       1003.0
99%       1003.0
max       1003.0
Name: description, dtype: float64

legalBasis_text length stats (chars):
count    27300.0
mean       356.0
std        198.0
min          3.0
50%        316.0
75%        401.0
90%        554.0
95%        678.0
99%       1038.0
max       9590.0
Name: legalBasis_text, dtype: float64

normalized_text length stats (chars):
count    27300.0
mean      1396.0
std        414.0
min          2.0
50%       1484.0
75%       1626.0
90%       1781.0
95%       1912.0
99%       2321.0
max      10864.0
Name: normalized_text, dtype: float64

Duplicated legislationIdentifier rows: 4000

Normalized_text > 50k chars: 0


In [ ]:
def build_embedding_text_act(row, max_chars=2000):
    name = str(row.get("name", "")).strip()
    desc = str(row.get("description", "")).strip()
    basis = str(row.get("legalBasis_text", "")).strip()
    ltype = str(row.get("legislationType", "")).strip()
    passed = str(row.get("legislationPassedBy", "")).strip()
    # Πάρε μόνο το πρώτο “useful” κομμάτι
    norm = str(row.get("normalized_text", "")).strip()
    norm_excerpt = norm[:max_chars]

    parts = [name, desc, ltype, passed, basis, norm_excerpt]
    return " ".join(p for p in parts if p)


In [ ]:
type(df.loc[0, "articles_raw"]), str(df.loc[0, "articles_raw"])[:120]


(str,
 "[{'article_number': '1', 'article_title': '', 'article_body': 'Annex VI to Regulation (EU) 2018/848 is replaced by the t")

In [ ]:
def build_embedding_text(row):
    parts = [
        str(row.get("name", "")),
        str(row.get("description", "")),
        str(row.get("legalBasis_text", "")),
        str(row.get("legislationType", "")),
        str(row.get("legislationPassedBy", ""))
    ]
    return " ".join(p for p in parts if p).strip()

df["embedding_text"] = df.apply(build_embedding_text, axis=1)
print(df["embedding_text"].head(2))


0    of on the European Commission becoming a partn...
1    (EU) 2020/1544 of amending Council Decision 20...
Name: embedding_text, dtype: object


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")
model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
def get_bert_embedding(texts, tokenizer, model, batch_size=16):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        encoded = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512)
        with torch.no_grad():
            output = model(**encoded)
        # Mean pooling
        last_hidden = output.last_hidden_state
        input_mask_expanded = encoded["attention_mask"].unsqueeze(-1).expand(last_hidden.size())
        sum_hidden = torch.sum(last_hidden * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        mean_pooled = sum_hidden / sum_mask
        embeddings.append(mean_pooled.cpu().numpy())
    return np.vstack(embeddings)

texts = df["embedding_text"].tolist()
embeddings = get_bert_embedding(texts, tokenizer, model)
print("Embedding shape:", embeddings.shape)


Embedding shape: (7117, 768)


In [ ]:
import os

save_dir = "/content/drive/MyDrive/LAWSKEPSIS/embeddings"
os.makedirs(save_dir, exist_ok=True)


In [ ]:
import numpy as np

def normalize(vectors):
    return vectors / np.linalg.norm(vectors, axis=1, keepdims=True)

embeddings_normed = normalize(embeddings)



In [ ]:
# Save embeddings
np.save("/content/drive/MyDrive/LAWSKEPSIS/embeddings/legalbert_embeddings.npy", embeddings_normed)

# Save corresponding internalIds (για να τα αναγνωρίζεις)
df[["internalId", "name"]].to_csv("/content/drive/MyDrive/LAWSKEPSIS/embeddings/internal_ids.csv", index=False)


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 78.5 MB/s eta 0:00:00


In [ ]:
import faiss

dimension = embeddings_normed.shape[1]
index = faiss.IndexFlatIP(dimension)  # IP = Inner Product → Cosine αν είναι normalized
index.add(embeddings_normed)

print(f"FAISS index contains {index.ntotal} embeddings")


FAISS index contains 7117 embeddings


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")
model.eval()

def encode_query(query):
    encoded = tokenizer(query, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        output = model(**encoded)
    last_hidden = output.last_hidden_state
    mask = encoded["attention_mask"].unsqueeze(-1).expand(last_hidden.size())
    pooled = torch.sum(last_hidden * mask, 1) / mask.sum(1)
    return normalize(pooled.cpu().numpy())

# def semantic_search(query, k=5):
#     query_vec = encode_query(query)
#     D, I = index.search(query_vec, k)
#     return df.iloc[I[0]][["internalId", "name", "description"]]


In [ ]:
def semantic_search(query, k=15):
    query_vec = encode_query(query)
    D, I = index.search(query_vec, k)

    results = df.iloc[I[0]].copy()
    results["similarity_score"] = D[0]  # cosine similarity
    return results[["internalId", "name", "description", "similarity_score"]].sort_values(by="similarity_score", ascending=False)


In [ ]:
top_results = semantic_search("EU company law framework for limited liability companies, including rules on formation, disclosure, capital maintenance, corporate governance and cross-border operations under Directive (EU) 2017/1132", k=10)
top_results.head(10)  # Εμφανίζει όλες τις στήλες για τα 5 πρώτα



,internalId,name,description,similarity_score
2197,2018/1212,"(EU) 2018/1212 of laying down minimum requirements implementing the provisions of Directive 2007/36/EC of the European Parliament and of the Council as regards shareholder identification, the transmission of information and the facilitation of the exercise of shareholders rights (Text with EEA relevance)","Directive 2007/36/EC gives the right to listed companies to identify their shareholders and requires intermediaries to cooperate in that identification process. That Directive also aims to improve the communication by listed companies to their shareholders, in particular the transmission of information along the chain of intermediaries and requires intermediaries to facilitate the exercise of shareholders rights. Those rights include the right to participate and vote in general meetings, and financial rights such as the right to receive the distributions of profits or participate in other corporate events initiated by the issuer or third party. This Regulation aims to prevent the diverging implementation of the provisions of Directive 2007/36/EC, which could result in the adoption of incompatible national standards, thereby increasing the risks and costs of cross-border operations and thus jeopardising their effectiveness and efficiency, and resulting in additional burdens for intermed...",0.910324
2591,2023/823,(EU) 2023/823 of laying down detailed rules for implementing certain provisions of Council Directive 2011/16/EU as regards the assessment and determination of equivalence of information in an agreement between the competent authorities of a Member State and a non-Union jurisdiction,"Directive 2011/16/EU was amended by Council Directive (EU) 2021/514 in order to improve the provisions that relate to all forms of exchanges of information and administrative cooperation by providing for a mandatory automatic exchange of information reported by platform operators. Given the nature and flexibility of digital platforms, the reporting obligation extends to those platform operators as defined in Annex V, Section IA, point 4(b) of Directive 2011/16/EU that perform commercial activity in the Union but are neither resident for tax purposes, nor incorporated or managed, nor have a permanent establishment in a Member State (foreign platform operators). This ensures a level playing field for all digital platform operators regardless of their place of establishment and prevents unfair competition within the Union. Directive 2011/16/EU lays down measures that are intended to reduce the administrative burden on foreign platform operators and the tax authorities of Member States, in...",0.906946
1817,2014/208,of on the quality of corporate governance reporting (comply or explain) (Text with EEA relevance) (2014/208/EU),"An effective corporate governance framework is of key importance to society, as well-run companies are likely to be more competitive and more sustainable in the long term. Good corporate governance is first and foremost the responsibility of the company concerned, and rules at European and national level are in place to ensure that certain standards are respected. These include legislation and soft law, namely national corporate governance codes. Corporate governance codes aim to establish principles for good corporate governance in listed companies in Europe based on transparency, accountability and a long-term perspective. They provide standards and best practice for companies, enabling them to perform better and therefore contribute to fostering growth, stability and long-term investment. Directive 2013/34/EU of the European Parliament and of the Council of on the annual financial statements, consolidated financial statements and related reports of certain types of undertakings requ...",0.905521
3015,2017/1542,(EU) 2017/1542 of amending Delegated Regulation (EU) 2015/35 concerning the calculation of regulatory capital requirements for certain categories of

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)


In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)

pd.set_option('display.max_columns', None)

pd.set_option('display.max_colwidth', None)

top_results


,internalId,name,description,similarity_score
5758,2014/135,of on a new approach to business failure and insolvency (Text with EEA relevance) (2014/135/EU),"The objective of this Recommendation is to ensure that viable enterprises in financial difficulties, wherever they are located in the Union, have access to national insolvency frameworks which enable them to restructure at an early stage with a view to preventing their insolvency, and therefore maximise the total value to creditors, employees, owners and the economy as a whole. The Recommendation also aims at giving honest bankrupt entrepreneurs a second chance across the Union. National insolvency rules vary greatly in respect of the range of the procedures available to debtors facing financial difficulties in order to restructure their business. Some Member States have a limited range of procedures meaning that businesses are only able to restructure at a relatively late stage, in the context of formal insolvency proceedings. In other Member States, restructuring is possible at an earlier stage but the procedures available are not as effective as they could be or involve varying degr...",0.844724
2591,2023/823,(EU) 2023/823 of laying down detailed rules for implementing certain provisions of Council Directive 2011/16/EU as regards the assessment and determination of equivalence of information in an agreement between the competent authorities of a Member State and a non-Union jurisdiction,"Directive 2011/16/EU was amended by Council Directive (EU) 2021/514 in order to improve the provisions that relate to all forms of exchanges of information and administrative cooperation by providing for a mandatory automatic exchange of information reported by platform operators. Given the nature and flexibility of digital platforms, the reporting obligation extends to those platform operators as defined in Annex V, Section IA, point 4(b) of Directive 2011/16/EU that perform commercial activity in the Union but are neither resident for tax purposes, nor incorporated or managed, nor have a permanent establishment in a Member State (foreign platform operators). This ensures a level playing field for all digital platform operators regardless of their place of establishment and prevents unfair competition within the Union. Directive 2011/16/EU lays down measures that are intended to reduce the administrative burden on foreign platform operators and the tax authorities of Member States, in...",0.840872
1817,2014/208,of on the quality of corporate governance reporting (comply or explain) (Text with EEA relevance) (2014/208/EU),"An effective corporate governance framework is of key importance to society, as well-run companies are likely to be more competitive and more sustainable in the long term. Good corporate governance is first and foremost the responsibility of the company concerned, and rules at European and national level are in place to ensure that certain standards are respected. These include legislation and soft law, namely national corporate governance codes. Corporate governance codes aim to establish principles for good corporate governance in listed companies in Europe based on transparency, accountability and a long-term perspective. They provide standards and best practice for companies, enabling them to perform better and therefore contribute to fostering growth, stability and long-term investment. Directive 2013/34/EU of the European Parliament and of the Council of on the annual financial statements, consolidated financial statements and related reports of certain types of undertakings requ...",0.837583
3316,2008/473,of concerning the limitation of the civil liability of statutory auditors and audit firms (notified under document number C(2008) 2274) (Text with EEA relevance) (2008/473/EC),"Directive 2006/43/EC of the European Parliament and of the Council of on statutory audits of annual accounts and consolidated accounts, amending Council Directives 78/660/EEC and 83/349

In [ ]:
import numpy as np
import pandas as pd

# Load embeddings
embeddings = np.load("/content/drive/MyDrive/LAWSKEPSIS/embeddings/legalbert_embeddings.npy")

# Load corresponding rows
df = pd.read_csv("/content/drive/MyDrive/LAWSKEPSIS/embeddings/metadata.csv")

assert len(df) == embeddings.shape[0], "Mismatch between df and embeddings!"


In [ ]:
# Με βάση το index:
D, I = index.search(query_vec, k=5)
results = df.iloc[I[0]][["internalId", "name", "description", "legislationPassedBy"]]
